---

_You are currently looking at **version 1.1** of this notebook. To download
notebooks and datafiles, as well as get help on Jupyter notebooks in the
Coursera platform, visit the [Jupyter Notebook
FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course
resource._

---

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments -
you are encouraged to check out the [pandas
documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions
or methods you might not have used yet, or ask questions on [Stack
Overflow](http://stackoverflow.com/) and tag them as pandas and python related.
And of course, the discussion forums are open for interaction with your peers
and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2
is April through June, Q3 is July through September, Q4 is October through
December.
* A _recession_ is defined as starting with two consecutive quarters of GDP
decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest
GDP.
* A _university town_ is a city which has a high percentage of university
students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by
recessions. Run a t-test to compare the ratio of the mean price of houses in
university towns the quarter before the recession starts compared to the
recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/)
there is housing data for the United States. In particular the datafile for [all
homes at a city
level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv),
`City_Zhvi_AllHomes.csv`, has median home sale prices at a fine grained
level.
* From the Wikipedia page on college towns is a list of [university towns in the
United
States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States)
which has been copy and pasted into the file `university_towns.txt`.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over
time](http://www.bea.gov/national/index.htm#gdp) of the United States in current
dollars (use the chained value in 2009 dollars), in quarterly intervals, in the
file `gdplev.xls`. For this assignment, only look at GDP data from the first
quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of
`run_ttest()`, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms:
states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AS': 'American Samoa',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'GU': 'Guam',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MP': 'Northern Mariana Islands',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NA': 'National',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'PR': 'Puerto Rico',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VI': 'Virgin Islands',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

In [3]:
def get_list_of_university_towns():
    """Return a DataFrame of towns and the states they are in."""

    # Read the text file, and load the records into a list:
    records = []
    with open('university_towns.txt', 'r') as txt_file:
        for line in txt_file:
            if '[edit]' in line:
                state = line.split('[')[0]
            else:
                town = line.split(' (')[0].strip()
                records.append([state, town])

    return pd.DataFrame(data=records, columns=['State', 'RegionName'])

In [4]:
# Read GDP data from 'gdplev.xls':
gdp = pd.read_excel(
    header=None,
    io='gdplev.xls',
    names=['Year Quarter', 'GDP Billions'],
    nrows=66,
    skiprows=220,
    usecols=[4, 6]
).set_index('Year Quarter')

# Add to 'gdp' two columns of Boolean values, 'Decline?' and 'Growth?'. For a
# given period, they tell us if there was GDP decline or growth with respect to
# the next period.
gdp['Decline?'] = gdp['GDP Billions'].gt(gdp['GDP Billions'].shift(-1))
gdp['Growth?'] = gdp['Decline?'].apply(lambda t: not t)

In [5]:
def get_recession_start():
    """Return year and quarter of the recession start."""
    return (gdp['Decline?'] & gdp['Decline?'].shift(-1)).astype('int64').shift(1).idxmax()

In [6]:
def get_recession_end():
    """Return year and quarter of the recession end."""
    tmp = gdp.loc[get_recession_start():, 'Growth?']
    return (tmp & tmp.shift(-1)).astype('int64').shift(2).idxmax()

In [7]:
def get_recession_bottom():
    """Return year and quarter of the recession bottom."""
    return gdp.loc[get_recession_start():get_recession_end(), 'GDP Billions'].idxmin()

In [8]:
def convert_housing_data_to_quarters():
    """Convert the housing data to quarters, and return it as mean values in a DataFrame."""

    # A few helper variables for dealing with the housing data:
    years = [str(year) for year in range(2000, 2017)]
    year_quarter = [y + 'q' + str(i) for y in years for i in range(1, 5)][:-1]
    months = ['{:02d}'.format(month) for month in range(1, 13)]
    year_month = [y + '-' + m for y in years for m in months][:-4]
    quarter_months = {year_quarter[i]: year_month[3 * i:3 * (i + 1)] for i in range(len(year_quarter))}
    with open('City_Zhvi_AllHomes.csv', 'r') as f:
        col_labels = f.readline().strip().split(',')
    col_labels = [''.join([char for char in label if char != '\"']) for label in col_labels]
    usecols = [1, 2] + [i for i in range(len(col_labels)) if col_labels[i] in year_month]

    # Read the housing data from 'City_Zhvi_AllHomes.csv':
    housing = pd.read_csv(
        'City_Zhvi_AllHomes.csv',
        header=0,
        usecols=usecols
    )

    housing['State'] = housing['State'].map(states)

    for yq, yms in quarter_months.items():
        housing[yq] = housing.loc[:, yms].mean(axis=1)
        housing.drop(columns=yms, inplace=True)
        #housing.drop(yms, axis=1, inplace=True)

    housing.set_index(['State', 'RegionName'], inplace=True)

    return housing

In [9]:
def run_ttest():

    housing = convert_housing_data_to_quarters()
    quarter_before_recession = housing.columns.get_loc(get_recession_start()) - 1
    price_ratio = housing.iloc[:, quarter_before_recession].div(housing[get_recession_bottom()])

    price_ratio_uni = pd.merge(
        get_list_of_university_towns(),
        price_ratio.reset_index(),
        on=['State', 'RegionName']
    ).set_index(['State', 'RegionName']).iloc[:, 0]
    price_ratio_non_uni = price_ratio.drop(index=price_ratio_uni.index)
    #price_ratio_non_uni = price_ratio.drop(price_ratio_uni.index, axis=0)

    ttest_res = ttest_ind(price_ratio_uni, price_ratio_non_uni, nan_policy='omit')
    p = ttest_res[1]
    different = True if p < 0.01 else False
    better = "university town" if price_ratio_uni.mean() < price_ratio_non_uni.mean() else "non-university town"
    return different, p, better

In [10]:
run_ttest()

(True, 0.002724063704753125, 'university town')